#
<FONT size="10pt"> <center>API N°5 : Création d'une API</center></FONT>. 

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import tqdm
import random
from skimpy import skim
import math

from IPython.display import display
pd.options.display.max_columns = None

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 1 : Récupération et nettoyage des données 

In [2]:
# Ouverture du fichier
Champignons = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/e5ba031c-ae1c-476a-a920-5fd401935b2a', encoding='latin1')
Champignons.head(3)

,Espece,Diametre minimum du chapeau,Diametre maximum du chapeau,Couleur du chapeau,Forme du chapeau,Surface du chapeau,Type de marge,Couleur de la marge,Dessous du chapeau,Couleur au dessous du chapeau,Oxydation du chapeau,Couleur de l oxydation du chapeau,Mode d insertion du dessous du chapeau avec le pied,Espace entre les lamelles,Type de lamelles,Couleur du pied,Pïed creux ou plein,Forme du pied,Longueur du pied,Surface su pied,Position du pied par rapport au chapeau,Presence d un anneau,Orientation de l anneau,Mobilite de l anneau,Couleur de l anneau,Poussee en touffe,Couleur de la base du pied,Couleur de la chair,Oxydation de la chair,Couleur de l oxydation,Fermete de la chair,Odeur,Latex,Couleur du latex,Mois de poussees le plus tot,Mois de poussees le plus tard,Milieu,Pousse sur du bois ou non,Nom vernaculaire,Comestibilite,Commentaires,Synonymes,Unnamed: 42
0,Acetabula vulgaris,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,absence d oxydation,NaN,NaN,NaN,NaN,NaN,pied plein,NaN,NaN,NaN,centrale,non,NaN,NaN,NaN,non,NaN,NaN,non,NaN,NaN,NaN,non,NaN,janvier ; janvier,decembre ; decembre,NaN,non,Pezize vulgaire,comestible,NaN,NaN,NaN
1,Agaricus abruptibulbus,8.0,12,blanc,NaN,ecailleuse ; mechuleuse ; squameuse,NaN,NaN,lamelles,gris ; rouge ; brun ; noir,absence d oxydation,NaN,libres,serrees,normal ;,blanc,pied creux,cylindrique ; tubulaire ; mince ; elance ; gre...,NaN,NaN,centrale,oui,descendant,NaN,blanc ;,non,NaN,blanc,oui,jaune,NaN,anis ; agreable,non,NaN,juin ; janvier,octobre ; decembre,coniferes,non,Psalliote a pied bulbeux,comestible,Peut etre confondu avec Agaricus silvicola (ps...,Agaricus essetei,NaN
2,Agaricus arvensis,3.0,15,blanc,ovoide ; ogival ; cylindrique ; campanule ; c...,ecailleuse ; mechuleuse ; squameuse ; lisse ; ...,NaN,NaN,lamelles,gris ; brun ; noir ; rose,absence d oxydation,jaune ;,libres,serrees,normal ;,blanc,pied plein,cylindrique ; tubulaire ; mince ; elance ; gre...,pied long,NaN,centrale,oui,descendant,NaN,blanc ;,non,NaN,blanc,oui,jaune,ferme,anis ; agreable,non,NaN,juin ; janvier,octobre ; decembre,prairies ; pres ; pelouses,non,Agaric des jacheres ; Boule de neige,bon comestible,Peut etre confondu avec Amanita virosa (amanit...,Psalliota arvensis,NaN


In [3]:
# Infos générales :
Champignons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 43 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Espece                                               1249 non-null   object 
 1   Diametre minimum du chapeau                          1246 non-null   float64
 2   Diametre maximum du chapeau                          1249 non-null   object 
 3   Couleur du chapeau                                   419 non-null    object 
 4   Forme du chapeau                                     448 non-null    object 
 5   Surface du chapeau                                   343 non-null    object 
 6   Type de marge                                        359 non-null    object 
 7   Couleur de la marge                                  31 non-null     object 
 8   Dessous du chapeau                                   948 non-null   

In [4]:
# Suppression de la colonne Unnamed :
del Champignons['Unnamed: 42']

In [5]:
# Description complète des données :
skim(Champignons)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1249   │ │ string      │ 41    │                                                          │
│ │ Number of columns │ 42     │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name                 ┃ NA   ┃ NA %   ┃ mean   ┃ sd    ┃ p0  ┃ p25   ┃ p50  ┃ p75  ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ Diametre minimum du chapeau │    3 │   0.24 │   0.92 │   1.9 │   0 │     0 │    0 │    0 │    10 │  ▇▁ ▁   │  │
│ └─────────────────────────────┴──────┴────────┴────────┴───────┴─────┴───────┴──────┴──────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                         ┃ NA       ┃ NA %       ┃ words per row           ┃ total words        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Espece                              │        0 │          0 │                       2 │               2510 │  │
│ │ Diametre maximum du chapeau         │        0 │          0 │                       1 │               1249 │  │
│ │ Couleur du chapeau                  │      830 │      66.45 │                     1.1 │               1388 │  │
│ │ Forme du chapeau                    │      801 │      64.13 │                     1.8 │               2264 │  │
│ │ Surface du chapeau                  │      906 │      72.54 │                     1.6 │               1956 │  │
│ │ Type de marge                       │      890 │      71.26 │                    0.95 │               1185 │  │
│ │ Couleur de la marge                 │     1218 │      97.52 │                    0.05 │                 62 │  │
│ │ Dessous du chapeau                  │      301 │       24.1 │                     1.1 │               1348 │  │
│ │ Couleur au dessous du chapeau       │      861 │      68.94 │                    0.91 │               1131 │  │
│ │ Oxydation du chapeau                │        3 │       0.24 │                       3 │               3739 │  │
│ │ Couleur de l oxydation du chapeau   │     1187 │      95.04 │                    0.11 │                141 │  │
│ │ Mode d insertion du dessous du      │      746 │      59.73 │                    0.89 │               1114 │  │
│ │ chapeau avec le pied                │          │            │                         │                    │  │
│ │ Espace entre les lamelles           │      980 │      78.46 │                    0.22 │                269 │  │
│ │ Type de lamelles                    │      637 │         51 │                       1 │               1276 │  │
│ │ Couleur du pied                     │      882 │      70.62 │                    0.85 │               1060 │  │
│ │ Pïed creux ou plein                 │       18 │    

In [6]:
# Transformation d'une colonne :
Champignons['Diametre maximum du chapeau'] = Champignons['Diametre maximum du chapeau'].apply(lambda x: float(x.replace(',','.')))

In [7]:
# Description d'une colonne :
print(min(Champignons['Diametre maximum du chapeau']))
print(max(Champignons['Diametre maximum du chapeau']))

0.0
99.0


# 2  : Création d'une API Fast sur Spyder et tests 

#### Commandes :
- uvicorn APIperso:app --reload
- http://127.0.0.1:8000/docs

In [8]:
# Lien url de la request : 
link = 'http://127.0.0.1:8000/champignons?diametre_max_chapeau=15&nombre_resultats=20'

# Request du lien : 
r = requests.get(link)

# Vérificiation de la bonne lecture de l'API : (200 = Good) :
r

# Lecture de la request en json :
ChapignonsAPI = r.json()

# Visualiser un fichier json de façon plus structuré :
ChapignonsAPI = pd.json_normalize(ChapignonsAPI)

# Data Frame :
ChapignonsAPI.head(3)

,Espece,Diametre minimum du chapeau,Diametre maximum du chapeau,Couleur du chapeau,Forme du chapeau,Surface du chapeau,Type de marge,Couleur de la marge,Dessous du chapeau,Couleur au dessous du chapeau,Oxydation du chapeau,Couleur de l oxydation du chapeau,Mode d insertion du dessous du chapeau avec le pied,Espace entre les lamelles,Type de lamelles,Couleur du pied,Pïed creux ou plein,Forme du pied,Longueur du pied,Surface su pied,Position du pied par rapport au chapeau,Presence d un anneau,Orientation de l anneau,Mobilite de l anneau,Couleur de l anneau,Poussee en touffe,Couleur de la base du pied,Couleur de la chair,Oxydation de la chair,Couleur de l oxydation,Fermete de la chair,Odeur,Latex,Couleur du latex,Mois de poussees le plus tot,Mois de poussees le plus tard,Milieu,Pousse sur du bois ou non,Nom vernaculaire,Comestibilite,Commentaires,Synonymes
0,Acetabula vulgaris,0.0,0.0,None,None,None,None,None,None,None,absence d oxydation,None,None,None,None,None,pied plein,None,None,None,centrale,non,None,None,None,non,None,None,non,None,None,None,non,None,janvier ; janvier,decembre ; decembre,None,non,Pezize vulgaire,comestible,None,None
1,Agaricus abruptibulbus,8.0,12.0,blanc,None,ecailleuse ; mechuleuse ; squameuse,None,None,lamelles,gris ; rouge ; brun ; noir,absence d oxydation,None,libres,serrees,normal ;,blanc,pied creux,cylindrique ; tubulaire ; mince ; elance ; gre...,None,None,centrale,oui,descendant,None,blanc ;,non,None,blanc,oui,jaune,None,anis ; agreable,non,None,juin ; janvier,octobre ; decembre,coniferes,non,Psalliote a pied bulbeux,comestible,Peut etre confondu avec Agaricus silvicola (ps...,Agaricus essetei
2,Agaricus arvensis,3.0,15.0,blanc,ovoide ; ogival ; cylindrique ; campanule ; c...,ecailleuse ; mechuleuse ; squameuse ; lisse ; ...,None,None,lamelles,gris ; brun ; noir ; rose,absence d oxydation,jaune ;,libres,serrees,normal ;,blanc,pied plein,cylindrique ; tubulaire ; mince ; elance ; gre...,pied long,None,centrale,oui,descendant,None,blanc ;,non,None,blanc,oui,jaune,ferme,anis ; agreable,non,None,juin ; janvier,octobre ; decembre,prairies ; pres ; pelouses,non,Agaric des jacheres ; Boule de neige,bon comestible,Peut etre confondu avec Amanita virosa (amanit...,Psalliota arvensis
